In [78]:
import pandas as pd

def unemployment_rate(year):
    # import US BLS labor stats; ignore noise rows at file start and improve column names
    df = pd.read_excel('./data/labor_force_xls/laucnty' + year + '.xlsx', header=4, usecols=["County Name/State Abbreviation","(%)"])
    df.columns = ("County","Percent Unemployed")

    # find only CT counties
    ct_mask = df["County"].str[-2:] == "CT"
    df = df[ct_mask]

    # strip to county name and set index
    df["County"] = df["County"].str[:-11]
    return df.set_index("County")
    
# to test
unemployment_rate('11')

,Percent Unemployed
County,
Fairfield,8.2
Hartford,9.0
Litchfield,8.1
Middlesex,7.6
New Haven,9.7
New London,8.9
Tolland,7.5
Windham,9.9
